# Twitter Trust Correlation

In [1]:
import pandas as pd
import numpy as np
import pickle as pk

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
raw_topics = pd.read_csv("https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/processed/article_topics_6.csv")
topics = raw_topics.set_index(['content_id'])
topics.head()

,Unnamed: 0,content_body_clean,content_title_clean,topic,topic_id1,topic_p1,topic_id2,topic_p2,topic_dist
content_id,,,,,,,,,
60,0,republican congress make break moment everyone...,tax reform failure act could mean minority sta...,Policy,0,0.548654,4.0,0.398622,"[(0, 0.54865676), (1, 0.035524607), (2, 0.0146..."
61,1,constitution authorize congress tax american p...,tax reform america need tax code good country ...,Policy,0,0.832200,3.0,0.098822,"[(0, 0.83220094), (3, 0.098824225), (4, 0.0664..."
66,2,longmothballed unfinished casinohotel la vega ...,longmothballed la vega strip casinohotel sell,Economy,2,0.677059,3.0,0.147280,"[(1, 0.09982404), (2, 0.6770535), (3, 0.147273..."
80,3,tax foreign earnings already accumulate overse...,trump tax plan may free corporate dollar,Policy,0,0.994644,NaN,NaN,"[(0, 0.9946441)]"
82,4,hurricane harvey already take enormous human t...,hurricane harvey humanitarian disaster also se...,Public Health,6,0.759787,2.0,0.237297,"[(2, 0.23729898), (6, 0.7597849)]"


In [3]:
raw_data = pd.read_csv("https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_articles.csv")
#raw_data.head()

In [22]:
articles = raw_data.set_index(['content_id'])
articles

,content_body_clean,content_title_clean,label,economy,topic2,science,mention_Trump,mention_Clinton,mention_Democrat,mention_Republican,...,mention_terrorist,mention_attack,mention_NRA,mention_sanctuary,mention_socialist,mention_fascist,mention_communist,mention_nationalist,mention_2A,content_source_desc
content_id,,,,,,,,,,,,,,,,,,,,,
60,Republicans in Congress are at a make or break...,Tax reform now -- failure to act could mean mi...,9,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,Fox News
61,The Constitution authorizes Congress to tax Am...,Tax reform -- America needs a tax code that's ...,9,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Fox News
66,"A long-mothballed, unfinished casino-hotel on ...",Long-mothballed Las Vegas Strip casino-hotel s...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fox News
80,If taxing foreign earnings that have already a...,"Trump Tax Plan May Free Up Corporate Dollars, ...",3,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,The New York Times
82,Hurricane Harvey has already taken an enormous...,Hurricane Harvey is a humanitarian disaster. I...,5,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Vox
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3380,"In this Oct. 17, 2017 file photo, Venezuela's...",Investors leave Venezuela meeting with no clea...,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Associated Press News
3381,A man watches stock prices boards at a brokera...,Global stocks turn lower after recent record h...,6,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Associated Press News
3382,The draining of a massive aquifer that underl...,Rural areas at risk as water levels drop in ma...,5,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Associated Press News


In [5]:
raw_urls = pd.read_csv("https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_urls.csv")
#raw_urls.head()

In [7]:
raw_tweets = pd.read_csv("http://muftayebaly.com/dropbox/tweetsCID.csv")
raw_tweets

,tweet_id,tweet_text,user_id,tweet_time,no_favs,no_retweets,urls,content_id
0,851485433003421701,#Trump’s border wall would be an ecological an...,613521891,2017-04-10T17:22:01.000Z,0,0,https://www.vox.com/energy-and-environment/201...,2736
1,851514117374398464,#Trump’s border wall would be an ecological an...,222481411,2017-04-10T19:16:00.000Z,0,0,https://www.vox.com/energy-and-environment/201...,2736
2,855009400670617600,The research and experts are clear: Tougher bo...,2347049341,2017-04-20T10:45:01.000Z,0,0,https://www.vox.com/policy-and-politics/2017/4...,2644
3,855009683542728704,Hmmmmmm https://t.co/y3fNx7KuJT,243580387,2017-04-20T10:46:08.000Z,0,0,https://www.vox.com/policy-and-politics/2017/4...,2644
4,855011873183342592,(psst! It was never about drugs. It's always b...,17889654,2017-04-20T10:54:50.000Z,0,0,https://www.vox.com/policy-and-politics/2017/4...,2644
...,...,...,...,...,...,...,...,...
201868,930566176907853824,Big increase in investment hastening arrival o...,454368263,2017-11-14T22:40:40.000Z,0,0,https://www.nytimes.com/2017/11/13/business/se...,3351
201869,930571261775876097,As readers of https://t.co/ajmt36bUQU already ...,61681958,2017-11-14T23:00:52.000Z,0,0,https://tytnetwork.com/Investigates/; https://...,3351
201870,930573435343908864,We’ve been reporting on what FedEx and UPS are...,905632398511046656,2017-11-14T23:09:30.000Z,0,0,https://www.nytimes.com/2017/11/13/business/se...,3351
201871,930582220800233473,Self-Driving Trucks May Be Closer Than They Ap...,831215040,2017-11-14T23:44:25.000Z,0,0,https://www.nytimes.com/2017/11/13/business/se...,3351


In [8]:
raw_tweets.describe()

,tweet_id,user_id,no_favs,no_retweets
count,2.018730e+05,2.018730e+05,201873.0,201873.0
mean,9.207283e+17,1.332248e+17,0.0,0.0
std,7.496690e+15,3.013590e+17,0.0,0.0
min,8.514854e+17,5.700000e+01,0.0,0.0
25%,9.174132e+17,5.645870e+07,0.0,0.0
50%,9.218926e+17,4.205027e+08,0.0,0.0
75%,9.260537e+17,2.853383e+09,0.0,0.0
max,9.305854e+17,9.301665e+17,0.0,0.0


In [4]:
raw_ratings = pd.read_csv("https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_ratings.csv")
raw_ratings.head()

,external_user_id,content_id,rating_scale_response,sampdate,blind
0,047641_S02_01,60,4.0,20170901,0
1,4205985366_78419_01,60,5.0,20170901,1
2,4205985366_78419_01,61,4.5,20170901,1
3,4205985366_78419_01,66,3.5,20170901,1
4,4205985366_78419_01,80,4.0,20170901,1


In [9]:
raw_users = pd.read_csv("http://muftayebaly.com/dropbox/all-user-stats.csv")
raw_users['content_id']=raw_tweets['content_id']
raw_users

,tweet_id,tweet_text,user_id,tweet_time,no_favs,no_retweets,user_friends,user_followers,user_statuses,user_listed,urls,content_id
0,851485433003421701,#Trump’s border wall would be an ecological an...,613521891,2017-04-10T17:22:01.000Z,0,0,1151,1514,2649,65,https://www.vox.com/energy-and-environment/201...,2736
1,851514117374398464,#Trump’s border wall would be an ecological an...,222481411,2017-04-10T19:16:00.000Z,0,0,3252,22924,13692,698,https://www.vox.com/energy-and-environment/201...,2736
2,855009400670617600,The research and experts are clear: Tougher bo...,2347049341,2017-04-20T10:45:01.000Z,0,0,212,570484,57251,12700,https://www.vox.com/policy-and-politics/2017/4...,2644
3,855009683542728704,Hmmmmmm https://t.co/y3fNx7KuJT,243580387,2017-04-20T10:46:08.000Z,0,0,300,6276,118715,257,https://www.vox.com/policy-and-politics/2017/4...,2644
4,855011873183342592,(psst! It was never about drugs. It's always b...,17889654,2017-04-20T10:54:50.000Z,0,0,457,1643,16647,80,https://www.vox.com/policy-and-politics/2017/4...,2644
...,...,...,...,...,...,...,...,...,...,...,...,...
201868,930566176907853824,Big increase in investment hastening arrival o...,454368263,2017-11-14T22:40:40.000Z,0,0,141,340,211,5,https://www.nytimes.com/2017/11/13/business/se...,3351
201869,930571261775876097,As readers of https://t.co/ajmt36bUQU already ...,61681958,2017-11-14T23:00:52.000Z,0,0,3190,3531,17059,121,https://tytnetwork.com/Investigates/; https://...,3351
201870,930573435343908864,We’ve been reporting on what FedEx and UPS are...,905632398511046656,2017-11-14T23:09:30.000Z,0,0,128,1555,115,19,https://www.nytimes.com/2017/11/13/business/se...,3351
201871,930582220800233473,Self-Driving Trucks May Be Closer Than They Ap...,831215040,2017-11-14T23:44:25.000Z,0,0,482,3903,9689,26,https://www.nytimes.com/2017/11/13/business/se...,3351


In [10]:
users = raw_users.set_index(['content_id'])

In [11]:
import os
import shutil

## Correlation of \# of Tweets and Trust

In [12]:
blind_ratings = raw_ratings[raw_ratings['blind']==1].groupby(['content_id']).mean()
blind_ratings['content_id1'] = blind_ratings.index
blind_ratings

,rating_scale_response,sampdate,blind,content_id1
content_id,,,,
60,5.000,20170901,1,60
61,4.500,20170901,1,61
66,3.500,20170901,1,66
80,3.750,20170901,1,80
82,3.500,20170901,1,82
...,...,...,...,...
3375,2.500,20170901,1,3375
3378,4.625,20170901,1,3378
3380,3.500,20170901,1,3380


In [16]:
num_tweets = raw_tweets.groupby(['content_id']).count()
num_tweets['content_id1'] = num_tweets.index
num_tweets

,tweet_id,tweet_text,user_id,tweet_time,no_favs,no_retweets,urls,content_id1
content_id,,,,,,,,
101,33,33,33,33,33,33,33,101
1019,14,14,14,14,14,14,14,1019
102,20,20,20,20,20,20,20,102
103,18,18,18,18,18,18,18,103
1060,2,2,2,2,2,2,2,1060
...,...,...,...,...,...,...,...,...
984,14,14,14,14,14,14,14,984
985,273,273,273,273,273,273,273,985
990,8,8,8,8,8,8,8,990


In [17]:
avg_users = users.groupby(['content_id']).mean()
avg_users

,tweet_id,user_id,no_favs,no_retweets,user_friends,user_followers,user_statuses,user_listed
content_id,,,,,,,,
101,9.022349e+17,5.155158e+16,0.0,0.0,7715.060606,18253.696970,117437.090909,554.000000
1019,9.051735e+17,2.458286e+17,0.0,0.0,9989.714286,36151.071429,111649.142857,886.285714
102,9.009004e+17,2.087483e+17,0.0,0.0,12668.350000,29445.750000,156883.850000,736.600000
103,8.987590e+17,1.873380e+17,0.0,0.0,7756.777778,42129.055556,98562.166667,1207.111111
1060,9.096514e+17,4.819011e+09,0.0,0.0,33.000000,203.500000,6953.000000,25.000000
...,...,...,...,...,...,...,...,...
984,9.091715e+17,1.619142e+17,0.0,0.0,4187.285714,19435.500000,104234.500000,270.071429
985,9.092689e+17,9.176621e+16,0.0,0.0,1828.959707,158635.344322,36624.619048,1064.611722
990,8.700455e+17,1.022339e+17,0.0,0.0,507.875000,76802.750000,20290.375000,1810.750000


In [31]:
med_users = users.groupby(['content_id']).median()
med_users

,tweet_id,user_id,no_favs,no_retweets,user_friends,user_followers,user_statuses,user_listed
content_id,,,,,,,,
101,9.022014e+17,7.255287e+07,0.0,0.0,1715.0,1520.0,39863.0,89.0
1019,9.051299e+17,9.552673e+08,0.0,0.0,2342.5,1396.0,18608.0,23.5
102,9.008499e+17,7.624337e+07,0.0,0.0,2334.0,1979.0,34406.0,52.5
103,8.987029e+17,6.219982e+08,0.0,0.0,1771.0,975.0,23858.0,20.0
1060,9.096514e+17,4.819011e+09,0.0,0.0,33.0,203.5,6953.0,25.0
...,...,...,...,...,...,...,...,...
984,9.091332e+17,1.316813e+09,0.0,0.0,584.0,1950.0,24393.5,59.5
985,9.092575e+17,2.684763e+08,0.0,0.0,648.0,675.0,8789.0,39.0
990,8.699896e+17,1.579408e+08,0.0,0.0,490.5,483.5,11670.0,31.5


In [32]:
corr_df = pd.DataFrame(columns=['content_id','article_body','article_title','num_tweets','trust_mean','source','topic',
                                'avg_friends','avg_followers','avg_statuses','avg_user_listed',
                               'med_friends','med_followers','med_statuses','med_user_listed'])

for index, row in num_tweets.iterrows():
  try:
    corr_df = corr_df.append({'content_id':index, 'article_body':articles['content_body_clean'][int(index)], 'article_title':articles['content_title_clean'][int(index)], 
                              'num_tweets':row['tweet_id'], 'trust_mean':blind_ratings['rating_scale_response'][int(index)], 
                              'source':articles['content_source_desc'][int(index)], 'topic':topics['topic'][int(index)], 
                              'avg_friends':avg_users['user_friends'][index], 'avg_followers':avg_users['user_followers'][index], 
                              'avg_statuses':avg_users['user_statuses'][index], 'avg_user_listed':avg_users['user_listed'][index],
                             'med_friends':med_users['user_friends'][index], 'med_followers':med_users['user_followers'][index], 
                              'med_statuses':med_users['user_statuses'][index], 'med_user_listed':med_users['user_listed'][index]}, ignore_index=True)
  except Exception:
    pass

corr_df

,content_id,article_body,article_title,num_tweets,trust_mean,source,topic,avg_friends,avg_followers,avg_statuses,avg_user_listed,med_friends,med_followers,med_statuses,med_user_listed
0,101,KELLY EVANS (CO-HOST): I just want to be clear...,CNBC lets Bush's disgraced FEMA director explo...,33,4.000000,Media Matters,Public Health,7715.060606,18253.696970,117437.090909,554.000000,1715.0,1520.0,39863.0,89.0
1,1019,Fox News used an unqualified crank to push dis...,"In honor of Labor Day, Fox & Friends let an ob...",14,1.000000,Media Matters,Economy,9989.714286,36151.071429,111649.142857,886.285714,2342.5,1396.0,18608.0,23.5
2,102,"Fox cut away from report on ""life-threatening""...","Fox cut away from report on ""life-threatening""...",20,0.500000,Media Matters,Public Health,12668.350000,29445.750000,156883.850000,736.600000,2334.0,1979.0,34406.0,52.5
3,103,Sarah Wasko / Media Matters Conservative ...,Business lobbying coalition recruits right-win...,18,0.500000,Media Matters,Politics,7756.777778,42129.055556,98562.166667,1207.111111,1771.0,975.0,23858.0,20.0
4,1060,A house rests on the beach after collapsing of...,Irma’s damage a reminder of Florida economy’s ...,2,4.000000,Associated Press News,Public Health,33.000000,203.500000,6953.000000,25.000000,33.0,203.5,6953.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,969,STEVE DOOCY (CO-HOST): It was a political expl...,"Fox & Friends lauds Trump's debt limit, spendi...",13,2.000000,Media Matters,Politics,2715.692308,26620.538462,48683.153846,795.769231,925.0,1104.0,40300.0,21.0
1251,984,Time is running out for Republicans if they wa...,Newt Gingrich: Tax cuts for small business wou...,14,4.166667,Fox News,Policy,4187.285714,19435.500000,104234.500000,270.071429,584.0,1950.0,24393.5,59.5
1252,985,"Starting with 1957, the team looked at actual ...",Bump in U.S. Incomes Doesn’t Erase 50 Years of...,273,2.750000,The New York Times,Public Health,1828.959707,158635.344322,36624.619048,1064.611722,648.0,675.0,8789.0,39.0
1253,990,The Associated Press reported today that Donal...,Critics say Trump’s cellphone diplomacy is dan...,8,2.800000,Vox,Technology,507.875000,76802.750000,20290.375000,1810.750000,490.5,483.5,11670.0,31.5


In [33]:
from scipy.stats.stats import pearsonr

In [34]:
corr_df1 = corr_df.astype({'num_tweets': 'int32'})

In [0]:
pearsonr(corr_df1['num_tweets'],corr_df1['trust_mean'])[0]

0.050122302409525775

In [0]:
np.corrcoef(corr_df1['num_tweets'].values.tolist(),corr_df1['trust_mean'].values.tolist())[0, 1]

0.05012230240952584

In [28]:
corr_df1

,content_id,article_body,article_title,num_tweets,trust_mean,source,topic,avg_friends,avg_followers,avg_statuses
0,101,KELLY EVANS (CO-HOST): I just want to be clear...,CNBC lets Bush's disgraced FEMA director explo...,33,4.000000,Media Matters,Public Health,7715.060606,18253.696970,117437.090909
1,1019,Fox News used an unqualified crank to push dis...,"In honor of Labor Day, Fox & Friends let an ob...",14,1.000000,Media Matters,Economy,9989.714286,36151.071429,111649.142857
2,102,"Fox cut away from report on ""life-threatening""...","Fox cut away from report on ""life-threatening""...",20,0.500000,Media Matters,Public Health,12668.350000,29445.750000,156883.850000
3,103,Sarah Wasko / Media Matters Conservative ...,Business lobbying coalition recruits right-win...,18,0.500000,Media Matters,Politics,7756.777778,42129.055556,98562.166667
4,1060,A house rests on the beach after collapsing of...,Irma’s damage a reminder of Florida economy’s ...,2,4.000000,Associated Press News,Public Health,33.000000,203.500000,6953.000000
...,...,...,...,...,...,...,...,...,...,...
1250,969,STEVE DOOCY (CO-HOST): It was a political expl...,"Fox & Friends lauds Trump's debt limit, spendi...",13,2.000000,Media Matters,Politics,2715.692308,26620.538462,48683.153846
1251,984,Time is running out for Republicans if they wa...,Newt Gingrich: Tax cuts for small business wou...,14,4.166667,Fox News,Policy,4187.285714,19435.500000,104234.500000
1252,985,"Starting with 1957, the team looked at actual ...",Bump in U.S. Incomes Doesn’t Erase 50 Years of...,273,2.750000,The New York Times,Public Health,1828.959707,158635.344322,36624.619048
1253,990,The Associated Press reported today that Donal...,Critics say Trump’s cellphone diplomacy is dan...,8,2.800000,Vox,Technology,507.875000,76802.750000,20290.375000


In [29]:
corr_df1.groupby(['source']).mean()

,num_tweets,trust_mean,avg_friends,avg_followers,avg_statuses
source,,,,,
100 Percent Fed Up,38.588235,2.222855,5078.596684,6941.932481,59601.539553
Associated Press News,4.295238,3.559105,2233.349159,48811.068800,36780.374397
Breitbart,130.113402,2.510586,4992.024438,10873.734905,74900.139012
Fox News,99.490196,3.264439,3049.896065,117718.596450,58662.712239
Media Matters,24.098039,2.572640,8431.892531,29686.102788,98459.964708
The New York Times,294.234177,3.555893,2613.652541,220921.438761,52926.282140
Vox,102.771552,3.373520,7254.542993,36208.125197,63829.565299


In [0]:
corr_df1.groupby(['topic']).mean()

,num_tweets,trust_mean,avg_friends,avg_followers,avg_statuses
topic,,,,,
Economy,87.508197,3.173621,2870.975037,161848.038625,65390.420461
Policy,183.082902,3.177887,4757.428486,88643.809879,60211.261361
Politics,123.334184,2.992726,4593.614341,100249.320937,63862.486464
Public Health,149.663830,3.543269,3773.961419,103533.417088,50560.743269
Scandal,166.871698,2.980684,4254.489063,85677.062380,62254.816997
Technology,42.541284,3.255277,4856.503906,40118.742417,63876.628291


In [0]:
print('100 Percent Fed Up:', pearsonr(corr_df[corr_df['source']=='100 Percent Fed Up'].num_tweets,
                                     corr_df[corr_df['source']=='100 Percent Fed Up'].trust_mean
                                     )[0])
print('Associated Press News:', pearsonr(corr_df[corr_df['source']=='Associated Press News'].num_tweets,
                                     corr_df[corr_df['source']=='Associated Press News'].trust_mean
                                     )[0])
print('Breitbart:', pearsonr(corr_df[corr_df['source']=='Breitbart'].num_tweets,
                                     corr_df[corr_df['source']=='Breitbart'].trust_mean
                                     )[0])
print('Fox News:', pearsonr(corr_df[corr_df['source']=='Fox News'].num_tweets,
                                     corr_df[corr_df['source']=='Fox News'].trust_mean
                                     )[0])
print('Media Matters:', pearsonr(corr_df[corr_df['source']=='Media Matters'].num_tweets,
                                     corr_df[corr_df['source']=='Media Matters'].trust_mean
                                     )[0])
print('The New York Times:', pearsonr(corr_df[corr_df['source']=='The New York Times'].num_tweets,
                                     corr_df[corr_df['source']=='The New York Times'].trust_mean
                                     )[0])
print('Vox:', pearsonr(corr_df[corr_df['source']=='Vox'].num_tweets,
                                     corr_df[corr_df['source']=='Vox'].trust_mean
                                     )[0])

100 Percent Fed Up: -0.029205437098217952
Associated Press News: -0.13419177533207938
Breitbart: -0.052556843882955256
Fox News: -0.27429417636187603
Media Matters: -0.05603416234001038
The New York Times: 0.08016191474413324
Vox: 0.03539024107968908


# Correlation of User Info and article trust

In [36]:
#print("User Avg Friends & Trust",pearsonr(corr_df1['avg_friends'],corr_df1['trust_mean'])[0])
#print("User Avg Followers & Trust",pearsonr(corr_df1['avg_followers'],corr_df1['trust_mean'])[0])
#print("User Avg User_Listed & Trust",pearsonr(corr_df1['avg_user_listed'],corr_df1['trust_mean'])[0])
#print()
print("User Median Friends & Trust",pearsonr(corr_df1['med_friends'],corr_df1['trust_mean'])[0])
print("User Median Followers & Trust",pearsonr(corr_df1['med_followers'],corr_df1['trust_mean'])[0])
print("User Median User_Listed & Trust",pearsonr(corr_df1['med_user_listed'],corr_df1['trust_mean'])[0])

User Avg Friends & Trust -0.10160187070124307
User Avg Followers & Trust 0.1506684903193611
User Avg User_Listed & Trust 0.17827870290140616

User Median Friends & Trust -0.1855593382579813
User Median Followers & Trust 0.020135757084879184
User Median User_Listed & Trust 0.04750796786740433


# Correlation Matrix

In [37]:
corrdf = pd.get_dummies(corr_df1, columns=['source','topic'])

In [38]:
corrdf.shape

(1255, 26)

In [41]:
corr_matrix = corrdf.corr()
corr_matrix

,num_tweets,trust_mean,avg_friends,avg_followers,avg_statuses,avg_user_listed,med_friends,med_followers,med_statuses,med_user_listed,...,source_Fox News,source_Media Matters,source_The New York Times,source_Vox,topic_Economy,topic_Policy,topic_Politics,topic_Public Health,topic_Scandal,topic_Technology
num_tweets,1.000000,0.050122,-0.115024,-0.002388,-0.157941,-0.058199,-0.106865,-0.062900,-0.141914,-0.075781,...,-0.062423,-0.075395,0.291990,-0.053837,-0.036659,0.062027,-0.031577,0.018197,0.048276,-0.094666
trust_mean,0.050122,1.000000,-0.101602,0.150668,-0.085013,0.178279,-0.185559,0.020136,-0.085159,0.047508,...,0.058746,-0.125152,0.244536,0.109797,0.004799,0.010956,-0.113356,0.195987,-0.093542,0.032919
avg_friends,-0.115024,-0.101602,1.000000,-0.063136,0.489171,0.044558,0.363563,-0.023451,0.342170,-0.022761,...,-0.128368,0.167165,-0.197613,0.275697,-0.065471,0.035867,0.034827,-0.053152,-0.008030,0.032002
avg_followers,-0.002388,0.150668,-0.063136,1.000000,0.007299,0.888359,-0.144075,0.070972,-0.058960,0.056603,...,0.062687,-0.068384,0.382404,-0.142127,0.079772,-0.011339,0.022626,0.024286,-0.021719,-0.085790
avg_statuses,-0.157941,-0.085013,0.489171,0.007299,1.000000,0.081965,0.430304,0.048063,0.778790,0.068534,...,-0.020979,0.172115,-0.097502,0.034488,0.024151,-0.003150,0.049299,-0.105719,0.019498,0.022655
avg_user_listed,-0.058199,0.178279,0.044558,0.888359,0.081965,1.000000,-0.157283,0.277448,0.005108,0.284927,...,-0.106152,0.001865,0.427728,-0.013964,0.092885,0.012171,-0.024611,0.074246,-0.066609,-0.052334
med_friends,-0.106865,-0.185559,0.363563,-0.144075,0.430304,-0.157283,1.000000,0.089412,0.630460,0.069827,...,-0.034696,0.024480,-0.203841,-0.152196,-0.018430,-0.067556,0.070652,-0.053902,0.068947,-0.040912
med_followers,-0.062900,0.020136,-0.023451,0.070972,0.048063,0.277448,0.089412,1.000000,0.164061,0.979403,...,-0.048142,-0.021227,-0.078548,-0.066075,-0.013371,-0.023095,0.047727,-0.019636,0.007763,-0.022799
med_statuses,-0.141914,-0.085159,0.342170,-0.058960,0.778790,0.005108,0.630460,0.164061,1.000000,0.188168,...,-0.015410,0.041897,-0.168890,-0.070601,0.035487,-0.055774,0.055208,-0.107596,0.034129,0.053078
med_user_listed,-0.075781,0.047508,-0.022761,0.056603,0.068534,0.284927,0.069827,0.979403,0.188168,1.000000,...,-0.059830,-0.027145,-0.071468,-0.053526,-0.010426,-0.027198,0.030985,0.003503,-0.009432,0.000625


In [48]:
corr_matrix[corr_matrix.columns[0]][1]
corr_matrix.index[1]

'trust_mean'

In [52]:
cor = pd.DataFrame(columns=['name','corr'])

size = corr_matrix.shape[0]
threshold = 0.17

for i in range(0,size):
    for j in range(0,size):
        name='';
        good=True
        if j < i:
            if abs(corr_matrix[corr_matrix.columns[i]][j]) > threshold:
                #remove bad ones
                if 'avg_' in corr_matrix.columns[i] or 'avg_' in corr_matrix.index[j]:
                    good=False
                if 'topic_' in corr_matrix.columns[i] and 'topic_' in corr_matrix.index[j]:
                    good=False
                if 'source_' in corr_matrix.columns[i] and 'source_' in corr_matrix.index[j]:
                    good=False
                
                if good:
                    name = corr_matrix.columns[i] + ' & ' + corr_matrix.index[j]
                    cor = cor.append({'name':name, 'corr':corr_matrix[corr_matrix.columns[i]][j]}, ignore_index=True)
        else:
            good=False
            break

In [53]:
cor

,name,corr
0,med_friends & trust_mean,-0.185559
1,med_statuses & med_friends,0.630460
2,med_user_listed & med_followers,0.979403
3,med_user_listed & med_statuses,0.188168
4,source_100 Percent Fed Up & trust_mean,-0.289807
5,source_100 Percent Fed Up & med_friends,0.287493
6,source_Associated Press News & med_followers,0.329862
7,source_Associated Press News & med_user_listed,0.376538
8,source_Breitbart & trust_mean,-0.287808
9,source_The New York Times & num_tweets,0.291990
